# Day 10
## Part 1
Represent lights and buttons with sets and BFS. You could do this more efficiently with bitmasks as the lights and buttons are binary numbers.

In [1]:
from advent import read_input

def parse_data(s):
    data = []
    for line in s.strip().splitlines():
        fields = line.strip().split()
        lights = frozenset(
            i 
            for i, c in enumerate(fields[0][1:-1]) 
            if c == "#"
        )
        buttons = [
            frozenset(eval(field.replace(")", ",)")))
            for field in fields[1:-1]
        ]
        joltages = eval(fields[-1].replace("{", "[").replace("}", "]"))
        data.append((lights, buttons, joltages))
    return data

test_data = parse_data("""[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
""")

test_data

[(frozenset({1, 2}),
  [frozenset({3}),
   frozenset({1, 3}),
   frozenset({2}),
   frozenset({2, 3}),
   frozenset({0, 2}),
   frozenset({0, 1})],
  [3, 5, 4, 7]),
 (frozenset({3}),
  [frozenset({0, 2, 3, 4}),
   frozenset({2, 3}),
   frozenset({0, 4}),
   frozenset({0, 1, 2}),
   frozenset({1, 2, 3, 4})],
  [7, 5, 12, 7, 2]),
 (frozenset({1, 2, 3, 5}),
  [frozenset({0, 1, 2, 3, 4}),
   frozenset({0, 3, 4}),
   frozenset({0, 1, 2, 4, 5}),
   frozenset({1, 2})],
  [10, 11, 11, 5, 10, 5])]

In [2]:
from collections import deque

def min_presses(lights, buttons):
    q = deque([(frozenset(), 0)])
    seen = {frozenset()}
    while q:
        l, n = q.popleft()
        if l == lights:
            return n
        for b in buttons:
            new_ls = l ^ b
            if new_ls not in seen:
                q.append((new_ls, n + 1))
                seen.add(new_ls)

[min_presses(ls, bs) for ls, bs, _ in test_data]

[2, 3, 2]

In [3]:
def part_1(data):
    return sum(min_presses(ls, bs) for ls, bs, _ in data)

assert part_1(test_data) == 7

In [4]:
data = parse_data(read_input())

part_1(data)

484

## Part 2

Good old dependable dynamic programming has failed me here, working on the test data but failing on the first instance in the real data.

These are a set of linear equations, where a given joltage is the sum of the number of buttons affecting that joltage is pressed. We would like to solve for each button's number of presses but this is not always possible as the number of unknowns can exceed the number of equations. So use Gaussian elimination for as many variables as possible and then search from there.

In [5]:
from fractions import Fraction
from copy import deepcopy

def create_eqs(buttons, joltages):
    matrix = []
    nb = len(buttons)
    for i, joltage in enumerate(joltages):
        matrix.append(
            [
                1 if i in buttons[j] else 0
                for j in range(len(buttons))
            ] + [joltage]
        )
    return matrix

def gaussian_elimination(matrix):
    w = len(matrix[0]) - 1
    h = len(matrix)
    d = min(w, h)
    for i in range(d):
        pivot_row = i
        while pivot_row < d - 1 and matrix[pivot_row][i] == 0:
            pivot_row += 1
        if pivot_row < h:
            if i != pivot_row:
                tmp = deepcopy(matrix[i])
                matrix[i] = matrix[pivot_row]
                matrix[pivot_row] = tmp
            pivot = matrix[i][i]
            if pivot != 0:
                matrix[i] = [Fraction(x, pivot) for x in matrix[i]]
                for j in range(i + 1, h):
                    matrix[j] = [x - matrix[j][i] * matrix[i][k] for k, x in enumerate(matrix[j])]
        # print_matrix(matrix)
        # print()
    return matrix[:d]

def print_matrix(m):
    for r in m:
        print([x.numerator if x.is_integer() else x for x in r])

In [6]:
m = create_eqs(data[1][1], data[1][2])
m


[[1, 1, 1, 0, 157],
 [1, 0, 0, 0, 11],
 [1, 1, 1, 0, 157],
 [0, 0, 1, 1, 141],
 [0, 1, 0, 1, 5]]

In [7]:
print_matrix(gaussian_elimination(m))

[1, 1, 1, 0, 157]
[0, 1, 1, 0, 146]
[0, 0, 1, 1, 141]
[0, 0, 0, 0, 0]


In [11]:
import math
import itertools

def calculate_presses(matrix):
    ns = {}
    for i in range(len(matrix) - 1, -1, -1):
        row = matrix[i]
        ns[i] = row[-1] - sum(ns[j] * row[j] for j in range(i + 1, len(matrix[0]) - 1))
    if all(n >= 0 and n.is_integer() for n in ns.values()):
        return sum(ns.values())
    else:
        return math.inf

def min_presses(buttons, joltages):
    matrix = create_eqs(buttons, joltages)
    matrix = gaussian_elimination(matrix)
    n_variables = len(matrix[0]) - 1
    n_equations = len(matrix)
    n_unknowns = n_variables - n_equations 
    presses = {}
    matrix.extend([[Fraction(0, 1)] * (n_variables + 1)] * n_unknowns)
    unknowns = [i for i, r in enumerate(matrix) if all(x == 0 for x in r)]
    for u in unknowns:
        matrix[u][u] = 1
    max_joltage = {
        u: sum(j for i, j in enumerate(joltages) if i in bs[u])
        for u in unknowns
    }
    min_so_far = math.inf
    for ns in itertools.product(*[range(max_joltage[u] + 1) for u in unknowns]):
        for i, n in zip(unknowns, ns):
            matrix[i][-1] = n
        min_so_far = min(min_so_far, calculate_presses(matrix))
    return min_so_far

In [12]:
for d in test_data:
    _, bs, js = d
    print(min_presses(bs, js))

72it [00:00, 11036.43it/s]


10


1it [00:00, 4739.33it/s]


12


23it [00:00, 28498.96it/s]

11


In [13]:
import tqdm

result = 0
for d in tqdm.tqdm(data):
    _, bs, js = d
    result += min_presses(bs, js)
result

  0%|                                             | 0/186 [00:00<?, ?it/s]
1it [00:00, 4405.78it/s]

147it [00:00, 22876.32it/s]

0it [00:00, ?it/s]
2177it [00:00, 21768.31it/s]
4505it [00:00, 22655.14it/s]
6837it [00:00, 22956.29it/s]
9178it [00:00, 23131.85it/s]
11644it [00:00, 23079.37it/s]
  2%|▌                                    | 3/186 [00:00<00:31,  5.75it/s]
0it [00:00, ?it/s]
4581it [00:00, 45804.91it/s]
9162it [00:00, 45330.89it/s]
13737it [00:00, 45518.87it/s]
19080it [00:00, 45583.64it/s]
  2%|▊                                    | 4/186 [00:00<00:46,  3.95it/s]
248it [00:00, 35393.77it/s]

0it [00:00, ?it/s]
6022it [00:00, 60211.01it/s]
12067it [00:00, 60349.42it/s]
18102it [00:00, 59880.82it/s]
24091it [00:00, 59713.39it/s]
31856it [00:00, 59448.25it/s]
  3%|█▏                                   | 6/186 [00:01<00:47,  3.81it/s]
1it [00:00, 21620.12it/s]

1it [00:00, 27060.03it/s]

250it [00:00, 27618.09it/s]

192it [00:00, 33218.10it/s]

0it [00:00, ?it/s]
3345it [00:00, 

KeyboardInterrupt: 